In [424]:
import json
import gurobipy as gp
from math import ceil

In [425]:
inst_name = "large"
with open(f"instances/{inst_name}.json", "r") as f:
    inst = json.load(f)

Pmax = inst["general_parameters"]["maximum_power"]
c0 = inst["general_parameters"]["curtailing_cost"]
cp = inst["general_parameters"]["curtailing_penalty"]
Cmax = inst["general_parameters"]["maximum_curtailing"]
cft = inst["general_parameters"]["fixed_cost_cable"]
clt = inst["general_parameters"]["variable_cost_cable"]
v0 = inst["general_parameters"]["main_land_station"]

cs = {v["id"]: v["cost"] for v in inst["substation_types"]}
rs = {v["id"]: v["rating"] for v in inst["substation_types"]}
ps = {v["id"]: v["probability_of_failure"] for v in inst["substation_types"]}

ls_cfq = {v["id"]: v["fixed_cost"] for v in inst["land_substation_cable_types"]}
ls_clq = {v["id"]: v["variable_cost"] for v in inst["land_substation_cable_types"]}
ls_pq = {v["id"]: v["probability_of_failure"] for v in inst["land_substation_cable_types"]}
ls_rq = {v["id"]: v["rating"] for v in inst["land_substation_cable_types"]}

ss_cfq = {v["id"]: v["fixed_cost"] for v in inst["substation_substation_cable_types"]}
ss_clq = {v["id"]: v["variable_cost"] for v in inst["substation_substation_cable_types"]}
ss_rq = {v["id"]: v["rating"] for v in inst["substation_substation_cable_types"]}

Vs = {v["id"]: {'x': v['x'], 'y': v['y']} for v in inst["substation_locations"]}
Vt = {v["id"]: {'x': v['x'], 'y': v['y']} for v in inst["wind_turbines"]}

S = cs.keys()
Q0 = ls_rq.keys()
Qs = ss_rq.keys()

In [426]:
def dist(u, v):
    return ((u["x"] - v["x"])**2 + (u["y"] - v["y"])**2)**0.5

In [427]:
# set E_0 of edges
E0 = {}
for id, v in Vs.items():
    E0[(0, id)] = dist(v0, v)
# set E_t of edges
Et = {}
for ids, s in Vs.items():
    for idt, t in Vt.items():
        Et[(ids, idt)] = dist(s, t)
# set E_s of edges
Es = {}
for ids1, s1 in Vs.items():
    for ids2, s2 in Vs.items():
        if ids1 < ids2:
            Es[(ids1, ids2)] = dist(s1, s2)

In [428]:
ss_ceq = {}
for e in Es:
    for q in Qs:
        ss_ceq[(e,q)] = ss_cfq[q] + ss_clq[q]*Es[e]
ls_ceq = {}
for e in E0:
    for q in Q0:
        ls_ceq[(e,q)] = ls_cfq[q] + ls_clq[q]*E0[e]

ce = {}
for e in Et:
    ce[e] = cft + clt*Et[e]

In [429]:
m = gp.Model(inst_name)
m.Params.TimeLimit = 30

Set parameter TimeLimit to value 30


In [430]:
x = {}
# creating variables x_{v,s} for each substation v each substation type s
for v in Vs:
    for s in S: # for each substation type
        x[(v,s)] = m.addVar(vtype=gp.GRB.BINARY, name=f"x_{v}{s}")
# adding constraint that each substation must be of at most one type
for v in Vs:
    m.addConstr(gp.quicksum(x[(v,s)] for s in S) <= 1, name=f"substation_type_{v}")

In [431]:
y = {}
# creating variables y_{e,q} for each edge e (between land and substations) and each cable type q
for e in E0:
    for q in Q0:
        y[(e,q)] = m.addVar(vtype=gp.GRB.BINARY, name=f"y_{e}{q}")
# adding constraint that each edge must be of exactly one type, if cable is used
for v in Vs:
    e = (0, v)
    m.addConstr(gp.quicksum(y[(e,q)] for q in Q0) == gp.quicksum(x[(v, s)] for s in S), name=f"edge_type_{e}")

In [432]:
z = {}
# creating variables z_e for each edge e (between substations and turbines)
for e in Et:
    z[e] = m.addVar(vtype=gp.GRB.BINARY, name=f"z_{e}")
# adding constraint that a single cable is built from each wind turbine
for t in Vt:
    Etv = [e for e in Et if e[1] == t]
    m.addConstr(gp.quicksum(z[e] for e in Etv) == 1, name=f"turbine_{t}")

In [433]:
# creating variables y_{e,q} for each edge e (between substation and substation) and each cable type q
for e in Es:
    for q in Qs:
        y[(e,q)] = m.addVar(vtype=gp.GRB.BINARY, name=f"y_{e}{q}")
# adding constraint that each built substation can be connected to at most 1 other substation
for v in Vs:
    Esv = [e for e in Es if e[0] == v or e[1] == v]
    m.addConstr(gp.quicksum(y[(e,q)] for e in Esv for q in Qs) <= gp.quicksum(x[(v,s)] for s in S), name=f"substation_{v}")

In [434]:
# add constraint that each substation must be of one type if it is connected to a turbine or another substation
for v in Vs:
    for e in Et:
        if e[0] == v:
            m.addConstr((z[e] == 1) >> (gp.quicksum(x[(v,s)] for s in S) >= 1), name=f"exists1_substation_{v}")
    for e in Es:
        if e[0] == v or e[1] == v:
            m.addConstr((y[(e,q)] == 1) >> (gp.quicksum(x[(v,s)] for s in S) >= 1), name=f"exists2_substation_{v}")

In [435]:
# add constraint that many substations must be used
for v in Vs:
    Etv = [e for e in Et if e[0] == v]
    m.addConstr(gp.quicksum(z[e] for e in Etv) <= ceil(len(Vt) / len(Vs)) + 120, name=f"many_substations_{v}")
for v in Vs:
    Etv = [e for e in Et if e[0] == v]
    for s in S:
        m.addConstr((x[v,s] == 1) >> (Pmax*gp.quicksum(z[e] for e in Etv) <= rs[s]))
for e in E0:
    v = e[1]
    Etv = [e for e in Et if e[0] == v]
    for q in Q0:
        m.addConstr((y[e,q] == 1) >> (Pmax*gp.quicksum(z[e] for e in Etv) <= ls_rq[q]))

In [436]:
# add a few interco
m.addConstr(gp.quicksum(y[(e,q)] for e in Es for q in Qs) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [437]:
# objective function
kappa = 0
kappa2 = 0
m.setObjective((
    gp.quicksum((cs[s]+kappa*rs[s]*ps[s])*x[(v,s)] for s in S for v in Vs) +
    gp.quicksum((ls_ceq[(e,q)]  + kappa2*ls_rq[q]*ls_pq[q])*y[(e,q)] for q in Q0 for e in E0) +
    gp.quicksum(ss_ceq[(e,q)]*y[(e,q)] for q in Qs for e in Es) +
    gp.quicksum(ce[e]*z[e] for e in Et)
), gp.GRB.MINIMIZE)

In [438]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 269 rows, 18655 columns and 49805 nonzeros
Model fingerprint: 0xd370bcd8
Model has 7350 general constraints
Variable types: 0 continuous, 18655 integer (18655 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
  GenCon rhs range [1e+00, 2e+03]
  GenCon coe range [1e+00, 2e+01]
Presolve added 5215 rows and 0 columns
Presolve removed 0 rows and 12495 columns
Presolve time: 1.65s
Presolved: 5484 rows, 6160 columns, 45220 nonzeros
Variable types: 0 continuous, 6160 integer (6125 binary)
Found heuristic solution: objective 10317.047342

Root relaxation: objective 2.874935e+03, 3370 iterations, 0.18 seconds (0.21 work units)

    Nodes 

In [439]:
substations = []
for (v, s), val in x.items():
    if val.x > 0.5:
        e = (0, v) # in E0
        for q in Q0:
            if y[(e, q)].x > 0.5:
                substations.append({
                    "id": v,
                    "substation_type": s,
                    "land_cable_type": q
                })

substation_substation_cables = []
for e in Es:
    for q in Qs:
        if y[(e, q)].x > 0.5:
            substation_substation_cables.append({
                "substation_id": e[0],
                "other_substation_id": e[1],
                "cable_type": q
            })

turbines = []
for (v, t), val in z.items():
    if val.x > 0.5:
        turbines.append({
            "id": t,
            "substation_id": v
        })

In [440]:
# write solution to file
with open(f"solutions/{inst_name}.json", "w") as f:
    json.dump({
        "substations": substations,
        "substation_substation_cables": substation_substation_cables,
        "turbines": turbines
    }, f, indent=4)